In [1]:
import pandas as pd
import numpy as np
from data import gen_binary_data
from protocols.grr import GRR_Client, GRR_Aggregator_MI, GRR_Aggregator_IBU
from sklearn.metrics import mean_squared_error
from protocols.L_GRR import L_GRR_Client, L_GRR_Aggregator_MI, L_GRR_Client_Interaction

In [176]:
n = 1000
data = gen_binary_data(n=n, one_prob=0.3)
k = 2
eps = [0.2, 0.5, 0.9]
num_seed = 10
real_freq = np.unique(data, return_counts=True)[-1] / n
dic_mse = {seed: {
    "GRR": []
} for seed in range(num_seed)}

dic_est_freq = {ep: {
    "GRR": []
} for ep in eps}

dic_reports = {ep: {
    "GRR": []
} for ep in eps}
print(f'Real frequency {real_freq}')


Real frequency [0.695 0.305]


In [177]:
# for seed in range(num_seed):
#     ep = eps[1]
#     grr_reports = [GRR_Client(input_data, k, ep) for input_data in data['v']]
#     grr_est_freq = GRR_Aggregator_IBU(grr_reports, k, ep)
#     print(f'Estimated frequency at seed: {seed}, epsilon: {ep}, ==> {grr_est_freq}')

#     dic_mse[seed]["GRR"].append(mean_squared_error(real_freq, grr_est_freq))
#     dic_est_freq[ep]["GRR"].append(grr_est_freq)
#     dic_reports[ep]["GRR"].append(np.unique(grr_reports, return_counts=True)[-1] / n)

# mean_errors = np.mean([dic_mse[seed]["GRR"] for seed in range(num_seed)], axis=0)

# print(f'Mean errors: {mean_errors}')

In [178]:
# initial_distn = np.mean(dic_est_freq[eps[1]]["GRR"], axis=0)
# initial_report = np.mean(dic_reports[eps[1]]["GRR"], axis=0)
# print(dic_est_freq)
# print(f'initial estimated distn: {initial_distn}')
# print(f'initial report: {initial_report}')




# 2nd iteration

In [179]:
# for seed in range(num_seed):
#     ep = eps[1]
#     grr_reports = [GRR_Client(input_data, k, eps[0]) if input_data == 1 
#                    else GRR_Client(input_data, k, eps[2]) for input_data in data['v']]
#     grr_est_freq = GRR_Aggregator_IBU(grr_reports, k, eps[2], True)
#     print(f'Estimated frequency at seed: {seed}, epsilon: {ep}, ==> {grr_est_freq}')

#     dic_mse[seed]["GRR"].append(mean_squared_error(real_freq, grr_est_freq))
#     dic_est_freq[ep]["GRR"].append(grr_est_freq)
#     dic_reports[ep]["GRR"].append(np.unique(grr_reports, return_counts=True)[-1] / n)

In [180]:
# initial_distn = np.mean(dic_est_freq[eps[1]]["GRR"], axis=0)
# initial_report = np.mean(dic_reports[eps[1]]["GRR"], axis=0)
# print(dic_est_freq)
# print(f'initial estimated distn: {initial_distn}')
# print(f'initial report: {initial_report}')


In [181]:
# def test_probs(epsilon):
#     p = np.exp(epsilon) / (np.exp(epsilon) + 1)
#     q = (1 - p)
#     return p,q

# eps = [0.01, 0.1, 0.2, 0.5, 0.9]

# for ep in eps:
#     p,q = test_probs(ep)
#     print(f'epsilon: {ep}, p: {p}, q:{q}')


In [182]:
eps_perm = 2
eps_1 = 0.2
k = 2
p1 = np.exp(eps_perm) / (np.exp(eps_perm) + k - 1)
q1 = (1 - p1) / (k - 1)

# GRR parameters for round 2
def get_second_eps(p1, q1, eps):
    p2 = (q1 - np.exp(eps) * p1) / (
                (-p1 * np.exp(eps)) + k * q1 * np.exp(eps) - q1 * np.exp(eps) - p1 * (k - 1) + q1)
    q2 = (1 - p2) / (k - 1)
    return p2, q2
p2, q2 = get_second_eps(p1, q1, eps_1)

print(p1, q1, p2, q2)
errors1 = []
for _ in range(num_seed):
    grr_reports = np.array([L_GRR_Client(input_data, k, eps_perm, eps_1) for input_data in data['v']]).astype(int)
    grr_est_freq = L_GRR_Aggregator_MI(grr_reports[:,1].tolist(), k, eps_perm, eps_1)
    errors1.append(mean_squared_error(real_freq, grr_est_freq))
    

0.8807970779778824 0.11920292202211757 0.5654337968887624 0.4345662031112376


In [183]:
permanent_randomized = grr_reports[:,0]
first_randomized = grr_reports[:,1]

print(f'Real freq: {np.unique(data, return_counts=True)[-1] / n}')
print(f'Permanent randomized freq: {np.unique(permanent_randomized, return_counts=True)[-1] / n}')
print(f'First randomized freq: {np.unique(first_randomized, return_counts=True)[-1] / n}')
print(f'First estimated freq: {grr_est_freq}')

Real freq: [0.695 0.305]
Permanent randomized freq: [0.652 0.348]
First randomized freq: [0.542 0.458]
First estimated freq: [0.92139907 0.07860093]


In [184]:
print(p1*p2+q1*q2)
print(p1*q2+q1*p2)
print((0.55*0.831) / (0.55 * 0.831 + 0.45 * 0.168))
print((0.55*0.80) / (0.55 * 0.80 + 0.45 * 0.20))


0.549833997312478
0.45016600268752205
0.8580681498169529
0.8301886792452831


In [185]:
eps_2_maj_x = 0.7
eps_2_min_x = eps_1

p3_maj, q3_maj = get_second_eps(p1, q1, eps_2_maj_x)
p3_min, q3_min = get_second_eps(p1, q1, eps_2_min_x)

eps_2_maj = np.log(p3_maj/q3_maj)
eps_2_min = np.log(p3_min/q3_min)

print(f'eps2maj: {eps_2_maj}, eps2min: {eps_2_min}, p3_maj {p3_maj}, q3_maj: {q3_maj}, p3_min {p3_min}, q3_min: {q3_min}')

print(p2 * p3_maj + q1 * q3_maj)
print(p1 * q3_min + q1 * p3_min)

print( (0.44 * 0.8)/ (0.44 * 0.8 + 0.4 * 0.2))
second_randomized = []
for inp in permanent_randomized:
    if inp == grr_est_freq.argmax():
        op = GRR_Client(inp, k, eps_2_maj)
    else:
        op = GRR_Client(inp, k, eps_2_min)
    second_randomized.append(op)


eps2maj: 0.9486146093690813, eps2min: 0.2632449193637755, p3_maj 0.7208364794463245, q3_maj: 0.27916352055367555, p3_min 0.5654337968887624, q3_min: 0.4345662031112376
0.44086241488124317
0.45016600268752205
0.8148148148148148


In [186]:


# print((0.55 * 0.8) + (0.45 * 0.2))
# print((0.44 * 0.8) + (0.4 * 0.2))

# print( (0.8 * 0.55 * 0.44)/ (0.432 * 0.53) )
errors = []
for seed in range(num_seed):
    majority_2_bin = []
    minority_2_bin = []
    for i in range(len(second_randomized)):
        first = first_randomized[i]
        second = second_randomized[i]

        # first = 0
        # second = 1
        majority = grr_est_freq.argmax()
        minority = grr_est_freq.argmin()
        p_b_majority = grr_est_freq.max()
        p_b_minority = grr_est_freq.min()
        

        p_1_majority_given_majority = p1 * p2 + q1 * q2
        p_1_majority_given_minority = p1 * q2 + q1 * p2
        p_1_minority_given_majority = p1 * q2 + q1 * p2
        p_1_minority_given_minority = p1 * p2 + q1 * q2
        
        p_2_majority_given_majority = p1 * p3_maj + q1 * q3_maj
        p_2_majority_given_minority = p1 * q3_min + q1 * q3_min
        p_2_minority_given_majority = p1 * q3_maj + q1 * p3_maj
        p_2_minority_given_minority = p1 * p3_min + q1 * q3_min


        p_b11_majority = p_2_majority_given_majority * p_b_majority + p_2_majority_given_minority * p_b_minority
        p_b1_majority = p_1_majority_given_majority * p_b_majority +  p_1_majority_given_minority * p_b_minority

        p_b11_minority = p_2_minority_given_majority * p_b_majority + p_2_minority_given_minority * p_b_minority
        p_b1_minority = p_1_minority_given_majority * p_b_majority + p_1_minority_given_minority * p_b_minority

        if first == majority and second == majority:
            # prob this user belongs to majority
            denominator = p_2_majority_given_majority * p_1_majority_given_majority * p_b_majority + p_2_majority_given_minority * p_1_majority_given_minority * p_b_minority
            p_this_user_majority = (p_b_majority * p_1_majority_given_majority * p_2_majority_given_majority)/ denominator
            # print(f'{p_b_majority} * {p_1_majority_given_majority} * {p_2_majority_given_majority} / {p_b11_majority} * {p_b1_majority}')
            # print(p_this_user_majority)

            # prob this user belongs to minority
            # p_this_user_minority = (p_b_minority * p_1_majority_given_minority * p_2_majority_given_minority)/ (p_b11_majority * p_b1_majority)
            # print(p_this_user_minority)
            # print(p_this_user_majority + p_this_user_minority)
        elif first == majority and second == minority:
            denominator = p_2_minority_given_majority * p_1_majority_given_majority * p_b_majority + p_2_minority_given_minority * p_1_majority_given_minority * p_b_minority
            p_this_user_majority = (p_b_majority * p_1_majority_given_majority * p_2_minority_given_majority)/ denominator
            # print(f'({p_b_majority} * {p_1_majority_given_majority} * {p_2_minority_given_majority})/ ({p_b11_minority} * {p_b1_majority})')
            # print(p_this_user_majority)

        elif first == minority and second == majority:
            denominator = p_2_majority_given_majority * p_1_minority_given_majority * p_b_majority + p_2_majority_given_minority * p_1_minority_given_minority * p_b_minority
            p_this_user_majority = (p_b_majority * p_1_minority_given_majority * p_2_majority_given_majority)/ (denominator)
            # print(f'{p_b_majority} * {p_1_minority_given_majority} * {p_2_majority_given_majority})/ ({p_b1_minority} * {p_b11_majority}')
            # print(p_this_user_majority)

        else: # minority and minority
            denominator = p_2_minority_given_majority * p_1_minority_given_majority * p_b_majority + p_2_minority_given_minority * p_1_minority_given_minority * p_b_minority
            p_this_user_majority = (p_b_majority * p_1_minority_given_majority * p_2_minority_given_majority) / (denominator)
            # print(f'{p_b_majority} * {p_1_minority_given_majority} * {p_2_minority_given_majority}) / ({p_b1_minority} * {p_b11_minority}')
            # print(p_this_user_majority)

        print('my p', first, second,  p_this_user_majority)
        if np.random.binomial(1, p_this_user_majority):
            # print('only here ', p_this_user_majority)
            majority_2_bin.append(second)
        else:
            # print('not here')
            minority_2_bin.append(second)

    l_grr_est_freq_majority_bin = L_GRR_Aggregator_MI(majority_2_bin, k, eps_perm, eps_2_maj_x)
    if minority_2_bin:
        l_grr_est_freq_minority_bin = L_GRR_Aggregator_MI(minority_2_bin, k, eps_perm, eps_2_min_x)
    else:
        minority_2_bin = np.zeros((2,))
    weighted_frequency = (l_grr_est_freq_majority_bin * len(majority_2_bin) + l_grr_est_freq_minority_bin * len(minority_2_bin)) / (len(majority_2_bin) + len(minority_2_bin))

    errors.append(mean_squared_error(real_freq, weighted_frequency))
    print(f'Seed: {seed}')


print(f'First error: {np.mean(errors1)}, second error: {np.mean(errors)}')
print(f'Real freq: {real_freq}, 1st estimation: {grr_est_freq}, 2nd estimation: {weighted_frequency}')


my p 0 1 0.8962711306083896
my p 1 0 0.9365369190357121
my p 1 0 0.9365369190357121
my p 1 1 0.8527660593137577
my p 0 1 0.8962711306083896
my p 0 1 0.8962711306083896
my p 0 1 0.8962711306083896
my p 1 1 0.8527660593137577
my p 0 0 0.9565503490813244
my p 0 1 0.8962711306083896
my p 0 1 0.8962711306083896
my p 1 1 0.8527660593137577
my p 0 0 0.9565503490813244
my p 0 0 0.9565503490813244
my p 1 0 0.9365369190357121
my p 1 0 0.9365369190357121
my p 0 0 0.9565503490813244
my p 1 1 0.8527660593137577
my p 0 0 0.9565503490813244
my p 0 0 0.9565503490813244
my p 0 0 0.9565503490813244
my p 1 0 0.9365369190357121
my p 0 0 0.9565503490813244
my p 0 1 0.8962711306083896
my p 1 0 0.9365369190357121
my p 1 1 0.8527660593137577
my p 1 1 0.8527660593137577
my p 0 1 0.8962711306083896
my p 1 1 0.8527660593137577
my p 1 1 0.8527660593137577
my p 1 1 0.8527660593137577
my p 0 1 0.8962711306083896
my p 0 0 0.9565503490813244
my p 1 0 0.9365369190357121
my p 1 0 0.9365369190357121
my p 0 1 0.896271130

In [187]:
# p1 , p3_maj , q1 , q3_maj, p3_min, q3_min
second_randomized

[1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,


In [188]:
print(p1 * p3_maj + q1 * q3_maj)
print(p1*q3_min + q1 * p3_min)

0.6681877721681662
0.45016600268752205


In [189]:
p_b_majority + p_b_minority

1.0

In [190]:
real_freq, grr_est_freq, weighted_frequency

(array([0.695, 0.305]),
 array([0.92139907, 0.07860093]),
 array([0.86528288, 0.13471712]))

In [6]:
eps_perm = 2
eps_1 = 1
k=2
p1 = np.exp(eps_perm) / (np.exp(eps_perm) + k - 1)
q1 = (1 - p1) / (k - 1)

        # GRR parameters for round 2
p2 = (q1 - np.exp(eps_1) * p1) / ((-p1 * np.exp(eps_1)) + k*q1*np.exp(eps_1) - q1*np.exp(eps_1) - p1*(k-1)+q1)
q2 = (1 - p2) / (k-1)

print(eps_perm, eps_1)

print(p1, q1, p2, q2)

e_sec = np.log(p2/q2)

print(e_sec)

p2_c = np.exp(e_sec) / (np.exp(e_sec) + k - 1)
q2_c = 1 - p2_c
print(p2_c, q2_c)
print(np.log(p1/q1))


2 1
0.8807970779778824 0.11920292202211757 0.803388066758518 0.19661193324148196
1.4076059644443795
0.803388066758518 0.19661193324148196
1.9999999999999998


In [43]:
a = []
b = np.array([2,3,4])
c = np.array([5,6,7])


print(b)
b = np.stack((b,c))
print(b)
print('--')
print(b[-1])

[2 3 4]
[[2 3 4]
 [5 6 7]]
--
[5 6 7]


In [69]:
# d = np.array([
#     [[1,2],[3,4]],
#     [[5,6],[7,8]],
#     [[9,10], [11,12]]
# ])

# n = 0
# user_prob = d[:,n]
# print(user_prob)
# print('---')

# print(np.prod(user_prob[:,1]))

p = np.array([0.5,0.2,0.6])
truth = np.random.binomial(1, p)
nr = np.array([0,1,2])

majority = 0
majority_reports = []
print(truth)
print(nr[truth == 1])

# a = np.array([ 
#     [ [1,2],[3,4] ],
#     [[10,20], [30,40]]
# ])

# for i in range(2):
#     arr = a[:,i]
#     print(arr)
#     print(np.mean(arr, axis=0))

1.2943226131421193e-05 < 4.012496053163908e-05

[0 1 1]
[1 2]


True

In [132]:
from scipy.stats import ks_2samp
from scipy.stats import kstest
from scipy.stats import anderson
from scipy.spatial import distance
import numpy as np
a = [1.9,0.8,0.94,0.8,0.91,0.96]
b = 1 * np.array([0.8,1110.94,11110.8,110.91,110.96, 122220.99])
b = 1 * np.array([0.8,0.94,0.8,0.91,0.96,1.4])

((5) * np.var(a, ddof=1)) / np.var(b, ddof=1)

17.940621419424414

In [102]:
np.var(b,ddof=1)

2408043483.0333467

In [4]:
import numpy as np
a = np.array([1,2,3,4,5])
b = np.array([0,0,1,0,1])

A = np.eye(2)
A[A == 1] = 0.1
A[A == 0] = 0.2
k=2
np.ones(k) / k

array([0.5, 0.5])

In [15]:
obs_freq = [0.6, 0.4]
est_freq = [0.5, 0.5]
# obs_freq / 
# est_freq *
np.dot(A, est_freq)


array([0.15, 0.15])

In [141]:
import random
n=10
random.sample(list(range(n)), int(n*0.2))

[1, 3]

In [143]:
import numpy as np
np.exp(0.1)/(np.exp(0.1)+1)

0.52497918747894